In [1]:
import pandas as pd
from mlxtend.frequent_patterns import association_rules, apriori
import time

In [2]:
df = pd.read_excel('HW4_dataset/transaction_dataset.xlsx')

In [3]:
df = df[df['QUANTITY']>0]
df.to_excel('HW4_dataset/transaction_dataset_after.xlsx')

In [4]:
df2 = df[["INVOICE_NO", "ITEM_ID"]]

In [5]:
h_data = {}
items = []
v_data = {}
for i in df.index:
    if df['ITEM_ID'][i] not in items:
        items.append(df['ITEM_ID'][i])
    if str(df['INVOICE_NO'][i]) in h_data:
        h_data[str(df['INVOICE_NO'][i])].append(str(df['ITEM_ID'][i]))
    else:
        h_data[str(df['INVOICE_NO'][i])] = [str(df['ITEM_ID'][i])]
        
    if str(df['ITEM_ID'][i]) in v_data:
        v_data[str(df['ITEM_ID'][i])].append(str(df['INVOICE_NO'][i]))
    else:
        v_data[str(df['ITEM_ID'][i])] = [str(df['INVOICE_NO'][i])]

In [6]:
class UtilityList:
    def __init__(self, item, UL):
        self.item = item
        self.UL = UL
        pass

In [7]:
def Construct(P_UL, Px_UL, Py_UL):
    Pxy_UL = []
    Ex_index = 0
    Ey_index = 0
    while Ex_index < len(Px_UL) and Ey_index < len(Py_UL):
        if Px_UL[Ex_index] == Py_UL[Ey_index]:
            Exy = []
            if P_UL is not None:
                for E in range(len(P_UL)):
                    if P_UL[E] == Px_UL[Ex_index]:
                        Exy = [Px_UL[Ex_index]]
            else:
                Exy = [Px_UL[Ex_index]]
            Pxy_UL.append(Exy)
            Ex_index += 1
            Ey_index += 1
        else:
            if Px_UL[Ex_index] > Py_UL[Ey_index]:
                Ey_index += 1
            else:
                Ex_index += 1
    return Pxy_UL

In [8]:
# brute force
def Construct2(P_UL, Px_UL, Py_UL):
    Pxy_UL = []
    for i in Px_UL:
        for j in Py_UL:
            if i == j:
                Pxy_UL.append(i)
    return Pxy_UL

In [9]:
def HUI_Miner(P, ULs, minutil):
    global frequent_itemsets
    for X in ULs:
        if len(X.UL) >= minutil:
            frequent_itemsets.append([len(X.UL)/len(h_data.keys()), [X.item[0]]])
            exULs = []
            for Y in ULs[ULs.index(X)+1:]:   
                X.item + [Y.item[-1]]
                exULs.append(UtilityList(X.item + [Y.item[-1]], Construct2(P.UL, X.UL, Y.UL)))
            HUI_Miner(X, exULs, minutil)

In [10]:
nn = []
for k, v in v_data.items():
    nn.append(UtilityList([k], list(set(v_data[k]))))

In [11]:
frequent_itemsets = []
HUI_Miner(UtilityList('', None), ULs=nn, minutil=len(h_data.keys())*0.001)

In [12]:
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(list(h_data.values())).transform(list(h_data.values()))
df2 = pd.DataFrame(te_ary, columns=te.columns_)
frequent_itemsets = apriori(df2, min_support=0.001, use_colnames=True)

rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

In [13]:
rules["antecedents"] = rules["antecedents"].apply(lambda x: ', '.join(list(x))).astype("unicode")
rules["consequents"] = rules["consequents"].apply(lambda x: ', '.join(list(x))).astype("unicode")

In [14]:
import xlsxwriter
workbook = xlsxwriter.Workbook('HW4_dataset/rule.xlsx')
worksheet = workbook.add_worksheet()
worksheet.write(0, 0, "antecedents")
worksheet.write(0, 1, "consequents")
row = 1
for i, j in zip(rules['antecedents'], rules['consequents']):
    worksheet.write(row, 0, i)
    worksheet.write(row, 1, j)
    row+=1
    
workbook.close()

In [15]:
df_rule = pd.read_excel('HW4_dataset/rule.xlsx')

In [16]:
def find_consequents(antecedents, df):
    li = []
    for k, v in zip(df_rule["antecedents"], df_rule["consequents"]):
        if antecedents in k:
            li.append(v)
    return list(set(li))
    

find_consequents("3336149, 88411", df_rule)

[70509]

In [17]:
min_support = [0.001, 0.0011, 0.0012, 0.0013, 0.0014]
rules_count = []
elapsed_time = []
for i in min_support:
    start_time = time.time()
    te = TransactionEncoder()
    te_ary = te.fit(list(h_data.values())).transform(list(h_data.values()))
    df2 = pd.DataFrame(te_ary, columns=te.columns_)
    frequent_itemsets = apriori(df2, min_support=i, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)
    elapsed_time.append(time.time()-start_time)
    rules_count.append(len(list(rules['antecedents'])))

In [18]:
rules_count, elapsed_time

([23, 12, 9, 3, 2],
 [20.032211542129517,
  13.355047941207886,
  10.13478136062622,
  7.137845039367676,
  5.370582342147827])

In [19]:
confidence = [0.5, 0.55, 0.6, 0.65, 0.7]
rules2_count = []
elapsed_time2 = []
workbook = xlsxwriter.Workbook('HW4_dataset/rule2.xlsx')
worksheet = workbook.add_worksheet()

for k in range(len(confidence)):
    start_time = time.time()
    te = TransactionEncoder()
    te_ary = te.fit(list(h_data.values())).transform(list(h_data.values()))
    df2 = pd.DataFrame(te_ary, columns=te.columns_)
    frequent_itemsets = apriori(df2, min_support=0.001, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=confidence[k])
    elapsed_time2.append(time.time()-start_time)
    rules2_count.append(len(list(rules['antecedents'])))
    
    
    rules["antecedents"] = rules["antecedents"].apply(lambda x: ', '.join(list(x))).astype("unicode")
    rules["consequents"] = rules["consequents"].apply(lambda x: ', '.join(list(x))).astype("unicode")
    worksheet.write(0, k*2, "antecedents"+str(confidence[k]))
    worksheet.write(0, k*2+1, "consequents"+str(confidence[k]))
    row = 1
    for i, j in zip(rules['antecedents'], rules['consequents']):
        worksheet.write(row, k*2, i)
        worksheet.write(row, k*2+1, j)
        row+=1
    
workbook.close()

In [20]:
rules2_count, elapsed_time2

([57, 48, 39, 31, 23],
 [20.06312870979309,
  19.94255566596985,
  19.709089279174805,
  19.85069966316223,
  19.988331079483032])

In [21]:
df_rule2 = pd.read_excel('HW4_dataset/rule2.xlsx')
df_rule2.head()

def find_consequents2(antecedents, df, confidence):
    li = []
    for i in confidence:
        li2 = []
        for k, v in zip(df["antecedents"+str(i)].dropna(), df["consequents"+str(i)].dropna()):
            if antecedents in k:
                li2.append(v)
        li.append(list(set(li2)))
    return li

confidence = [0.5, 0.55, 0.6, 0.65, 0.7]
product_count = find_consequents2("70509", df_rule2, confidence)
print(product_count)
print("Product count:", [len(x) for x in product_count])

[['135493', '3336149', '1698', '23004', '88444', '1708', '1697', '88411'], ['1698', '88444', '1708', '1697', '88411'], ['1708', '1698', '88411', '88444'], ['1708', '88411'], [88411.0]]
Product count: [8, 5, 4, 2, 1]


In [22]:
from mlxtend.frequent_patterns import fpgrowth

frequent_itemsets = fpgrowth(df2, min_support=0.001, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)

In [23]:
min_support = [0.001, 0.0011, 0.0012, 0.0013, 0.0014]
rules_count = []
elapsed_time = []
for i in min_support:
    start_time = time.time()
    te = TransactionEncoder()
    te_ary = te.fit(list(h_data.values())).transform(list(h_data.values()))
    df2 = pd.DataFrame(te_ary, columns=te.columns_)
    frequent_itemsets = fpgrowth(df2, min_support=i, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)
    elapsed_time.append(time.time()-start_time)
    rules_count.append(len(list(rules['antecedents'])))

In [28]:
rules_count, elapsed_time

([57, 48, 39, 31, 23],
 [1.66853928565979,
  1.676504373550415,
  1.6884393692016602,
  1.6844992637634277,
  1.7363154888153076])

In [25]:
confidence = [0.5, 0.55, 0.6, 0.65, 0.7]
rules_count = []
elapsed_time = []
for i in confidence:
    start_time = time.time()
    te = TransactionEncoder()
    te_ary = te.fit(list(h_data.values())).transform(list(h_data.values()))
    df2 = pd.DataFrame(te_ary, columns=te.columns_)
    frequent_itemsets = fpgrowth(df2, min_support=0.001, use_colnames=True)
    rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=i)
    elapsed_time.append(time.time()-start_time)
    rules_count.append(len(list(rules['antecedents'])))

In [29]:
rules_count, elapsed_time

([57, 48, 39, 31, 23],
 [1.66853928565979,
  1.676504373550415,
  1.6884393692016602,
  1.6844992637634277,
  1.7363154888153076])